## Environment creation:

```sh
mamba create -n ibis-substrait-duckdb python=3.10 ibis-duckdb ibis-substrait python-duckdb duckdb-engine=0.1.10 jupyter
conda activate ibis-substrait-duckdb

# install latest dev version of duckdb
python -m pip install -U duckdb==0.4.0
```

In [1]:
import ibis
import ibis.expr.datatypes as dt
from datetime import date
from pathlib import Path

In [2]:
print(ibis.__version__)

3.0.2


`ibis-substrait` compiler

In [3]:
import ibis_substrait
from ibis_substrait.compiler.core import SubstraitCompiler

print(ibis_substrait.__version__)

2.9.2


`duckdb` Python wrapper

In [4]:
import duckdb

print(duckdb.__version__)

0.4.0


### Create `TPC-H` sample data with scale factor of 0.1 for demo

In [5]:
Path("tpch.ddb").unlink(missing_ok=True)

In [6]:
con = duckdb.connect("tpch.ddb")

In [7]:
con.execute("CALL dbgen(sf=0.1)")

### Install and load `duckdb` `substrait` extension

In [8]:
con.execute("install substrait")

In [9]:
con.execute("load substrait")

Load `ibis` and create unbound `TPC-H` tables

In [10]:
lineitem = ibis.table(
    [
        ("l_orderkey", dt.int64),
        ("l_partkey", dt.int64),
        ("l_suppkey", dt.int64),
        ("l_linenumber", dt.int64),
        ("l_quantity", dt.Decimal(15, 2)),
        ("l_extendedprice", dt.Decimal(15, 2)),
        ("l_discount", dt.Decimal(15, 2)),
        ("l_tax", dt.Decimal(15, 2)),
        ("l_returnflag", dt.string),
        ("l_linestatus", dt.string),
        ("l_shipdate", dt.date),
        ("l_commitdate", dt.date),
        ("l_receiptdate", dt.date),
        ("l_shipinstruct", dt.string),
        ("l_shipmode", dt.string),
        ("l_comment", dt.string),
    ],
    name="lineitem",
)

In [11]:
lineitem

UnboundTable: lineitem
  l_orderkey      int64
  l_partkey       int64
  l_suppkey       int64
  l_linenumber    int64
  l_quantity      decimal(prec=15, scale=2)
  l_extendedprice decimal(prec=15, scale=2)
  l_discount      decimal(prec=15, scale=2)
  l_tax           decimal(prec=15, scale=2)
  l_returnflag    string
  l_linestatus    string
  l_shipdate      date
  l_commitdate    date
  l_receiptdate   date
  l_shipinstruct  string
  l_shipmode      string
  l_comment       string

TPC-H Query 1 in Ibis

In [12]:
tpc_h1_expr = (
    lineitem.filter(lambda t: t.l_shipdate <= date(year=1998, month=9, day=2))
    .group_by(["l_returnflag", "l_linestatus"])
    .aggregate(
        sum_qty=lambda t: t.l_quantity.sum(),
        sum_base_price=lambda t: t.l_extendedprice.sum(),
        sum_disc_price=lambda t: (t.l_extendedprice * (1 - t.l_discount)).sum(),
        sum_charge=lambda t: (
            t.l_extendedprice * (1 - t.l_discount) * (1 + t.l_tax)
        ).sum(),
        avg_qty=lambda t: t.l_quantity.mean(),
        avg_price=lambda t: t.l_extendedprice.mean(),
        avg_disc=lambda t: t.l_discount.mean(),
        count_order=lambda t: t.count(),
    )
    .sort_by(["l_returnflag", "l_linestatus"])
)

In [13]:
tpc_h1_expr

r0 := UnboundTable: lineitem
  l_orderkey      int64
  l_partkey       int64
  l_suppkey       int64
  l_linenumber    int64
  l_quantity      decimal(prec=15, scale=2)
  l_extendedprice decimal(prec=15, scale=2)
  l_discount      decimal(prec=15, scale=2)
  l_tax           decimal(prec=15, scale=2)
  l_returnflag    string
  l_linestatus    string
  l_shipdate      date
  l_commitdate    date
  l_receiptdate   date
  l_shipinstruct  string
  l_shipmode      string
  l_comment       string

r1 := Aggregation[r0]
  metrics:
    sum_qty:        Sum(r0.l_quantity)
    sum_base_price: Sum(r0.l_extendedprice)
    sum_disc_price: Sum(r0.l_extendedprice * 1 - r0.l_discount)
    sum_charge:     Sum(r0.l_extendedprice * 1 - r0.l_discount * r0.l_tax + 1)
    avg_qty:        Mean(r0.l_quantity)
    avg_price:      Mean(r0.l_extendedprice)
    avg_disc:       Mean(r0.l_discount)
    count_order:    Count(lineitem)
  by:
    l_returnflag: r0.l_returnflag
    l_linestatus: r0.l_linestatus
  predicat

Import `ibis-subtrait` compiler to serialize `ibis` expression to `substrait`

In [14]:
compiler = SubstraitCompiler()

In [15]:
tpch_proto_bytes = compiler.compile(tpc_h1_expr)

In [16]:
tpch_proto_bytes

extension_uris {
  extension_uri_anchor: 1
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 1
    name: "lte"
  }
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 2
    name: "sum"
  }
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 3
    name: "multiply"
  }
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 4
    name: "subtract"
  }
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 5
    name: "add"
  }
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 6
    name: "mean"
  }
}
extensions {
  extension_function {
    extension_uri_reference: 1
    function_anchor: 7
    name: "count"
  }
}
relations {
  root {
    input {
      sort {
        input {
          aggregate {
            input {
              filter {
                input {
       

Now let's return to our `duckdb` connection

In [17]:
con

In [18]:
results_substrait = con.from_substrait(tpch_proto_bytes.SerializeToString())

In [19]:
results_substrait_df = results_substrait.to_df()
results_substrait_df

,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,3774200.0,5.320754e+09,5.054096e+09,5.256751e+09,25.537587,36002.123829,0.050145,147790
1,N,F,95257.0,1.337378e+08,1.271324e+08,1.322863e+08,25.300664,35521.326916,0.049394,3765
2,N,O,7459297.0,1.051227e+10,9.986238e+09,1.038558e+10,25.545538,36000.924688,0.050096,292000
3,R,F,3785523.0,5.337951e+09,5.071819e+09,5.274406e+09,25.525944,35994.029214,0.049989,148301


Now delete the `duckdb` con and let's re-run the query using the `ibis.duckdb` backend to compare results

In [20]:
del con

Create `ibis-duckdb` connection

In [21]:
con = ibis.duckdb.connect("tpch.ddb")

In [22]:
results_ibis_duckdb = con.execute(tpc_h1_expr)

/home/gil/mambaforge/envs/ibis-substrait-duckdb/lib/python3.10/site-packages/ibis/backends/base/sql/__init__.py:95: SAWarning: Dialect duckdb+psycopg2 does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  cursor = self.con.execute(query)  # type: ignore


In [23]:
results_ibis_duckdb

,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,3774200.0,5320753880.69,5054096266.6828,5256751331.449234,25.537587,36002.123829,0.050145,147790
1,N,F,95257.0,133737795.84,127132372.6512,132286291.229445,25.300664,35521.326916,0.049394,3765
2,N,O,7459297.0,10512270008.9,9986238338.384701,10385578376.585468,25.545538,36000.924688,0.050096,292000
3,R,F,3785523.0,5337950526.47,5071818532.942,5274405503.049367,25.525944,35994.029214,0.049989,148301


Minor annoyance that the `ibis` results come back as `object` type and `duckdb` comes back as `float64` -- quick `astype` cast before value comparison

In [24]:
results_ibis_duckdb = results_ibis_duckdb.astype(
    {
        "sum_qty": "float64",
        "sum_base_price": "float64",
        "sum_disc_price": "float64",
        "sum_charge": "float64",
        "avg_qty": "float64",
        "avg_price": "float64",
        "avg_disc": "float64",
    }
)

In [25]:
results_ibis_duckdb.equals(results_substrait_df)

True